## pickle all the things

we scan all the images, and run them through the pipeline to create tensors. 
We than save that as a .pkl file for later usage



In [ ]:
!pip install -q optimum[onnxruntime]==1.23.3 transformers==4.46.3 torch==2.2.2+cu121
import time
import requests

In [ ]:
from optimum.onnxruntime import ORTModelForImageClassification
from transformers import AutoFeatureExtractor

# Load and export the model to ONNX
model = ORTModelForImageClassification.from_pretrained("rh-ai-bu/wildfire01", export=True)

# Load the feature extractor
feature_extractor = AutoFeatureExtractor.from_pretrained("rh-ai-bu/wildfire01")

# Create an ONNX-based pipeline
from optimum.pipelines import pipeline
onnx_pipeline = pipeline("image-classification", model=model, feature_extractor=feature_extractor, accelerator="ort")

In [ ]:
onnx_pipeline.save_pretrained("onnx_pipeline")

In [ ]:

# with open('var.pkl', 'rb') as file:  
#     image_preprocessed = pickle.load(file)
#     print(image_preprocessed)  # This will print: [1, 2, 3, 'Hello', [4, 5]]

# # Open the file for writing
# with open('var.pkl', 'wb') as file:  # 'wb' stands for write in binary mode
#     pickle.dump(image_preprocessed, file)


In [ ]:
import os
import glob
import pickle
import time
import requests

def convert_and_pickle(path='/opt/app-root/src/ai-mazing-race/lab-materials/06/06-wildfire-sample'):
    # List of image file extensions
    image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.gif']

    # Create a list to hold full paths of image files
    image_files = []
    processed_files = 0  # Initialize a counter for processed files


    # Iterate over each file extension
    for ext in image_extensions:
        # Use glob to find all files with the current extension in the given path and its subdirectories
        for file in glob.iglob(os.path.join(path, '**', ext), recursive=True):
            # Append the full path to the list
            image_files.append(file)
            if processed_files >= 50:  # Check if we've processed 50 files
                break  # If so, exit the loop
            processed_files += 1  # Increment the counter

    # Print each image file's full path
    for file in image_files:
        # print(file)
        mytensors = onnx_pipeline.preprocess(file)
        
        base_name = os.path.basename(file)
        newfile = os.path.join(os.path.dirname(file), os.path.splitext(base_name)[0] + ".pkl")
        # print(newfile)
        # Open the file for writing
        with open(newfile, 'wb') as file:
            pickle.dump(mytensors, file)        

# Call the function
convert_and_pickle()
